In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from RCN_alex import *

# Part 4

## Q1

Estimate $\delta$ and $r$.

Load price list

In [2]:
data = pd.read_csv('Data-Project1-Fin404.csv')
data

,Strike,Call,Put
0,12000,381.525,1662.321
1,11800,420.599,1499.822
2,11600,493.332,1370.990
3,11400,571.380,1247.470
4,11200,649.428,1123.951
5,11000,727.476,1000.440
6,10800,805.524,876.917
7,10600,883.572,753.399
8,10400,991.344,659.605
9,10200,1114.862,581.557


Linear regression of call-put against strike

In [3]:
ols = sm.OLS(
    data.Call - data.Put,
    sm.add_constant(data.Strike)
).fit()
alpha, beta = ols.params

Derive values for $\delta$ and $r$

In [4]:
i0 = 11118
r = -np.log(-beta)
delta = -np.log(alpha/i0)

print(
"""
r: {:.2%}%
delta: {:.2%}%
""".format(r, delta)
)


r: -0.78%%
delta: 2.78%%



## Q2

Calibrate the binomial model.

Define the value of the other know parameters.

In [5]:
# Other params
Delta = 30/360
q = 1/2
T = 12

In [6]:
# Us = np.arange(1, np.exp(Delta * r) / q, .0001)

# calibrator = Binomial_calibrator(r, Delta, i0, delta, Us, q, T, data.Strike, data.Call)

# best_U, best_D = calibrator.calibrate()

In [7]:
best_U, best_D = 1.05500, 0.94370

# Part 5

## Q1

### i)

In [8]:
RCN_binomial(r, Delta, i0, delta, best_U, best_D, range(1, 12), 0.1, exercise_price=1, barrier_level=None, _callable=False).price_RCN()

1.0099364617538527

### ii)

In [9]:
RCN_binomial(r, Delta, i0, delta, best_U, best_D, range(1, 12), 0.1, exercise_price=1, barrier_level=0.8, _callable=False).price_RCN()

1.063343444651538

Old model with no + Delta c on not callable

In [7]:
# RCN_binomial(r, Delta, i0, delta, best_U, best_D, range(1, 12), 0.1, exercise_price=1, barrier_level=None, _callable=False).price_RCN()

1.0099364716992132

In [8]:
# RCN_binomial(r, Delta, i0, delta, best_U, best_D, range(1, 12), 0.1, exercise_price=1, barrier_level=0.8, _callable=False).price_RCN()

1.0633435820497554

## Q2

In [ ]:
def compute_par_coupon_Rate(alpha, beta):
    cs = range(0.001, 0.20, 0.001)
    for c in cs:
        RCN_binomial(r, Delta, i0, delta, best_U, best_D, range(1, 12), c, exercise_price=1, barrier_level=0.8, _callable=False).price_RCN()

Create a binomial model with the computed parameters.

In [ ]:
model = RCN_binomial(r, Delta, i0, delta, best_U, best_D, dates, c, alpha, barrier_level=None, _callable=False)